In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('C:/Users/DINH QUOC BAO/Documents/data_toxic/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:/Users/DINH QUOC BAO/Documents/data_toxic/input\sample_submission.csv.zip
C:/Users/DINH QUOC BAO/Documents/data_toxic/input\test.csv.zip
C:/Users/DINH QUOC BAO/Documents/data_toxic/input\test_labels.csv.zip
C:/Users/DINH QUOC BAO/Documents/data_toxic/input\train.csv.zip


In [12]:
# dataset_directory = 'C:/Users/DINH QUOC BAO/Documents/data_toxic/input'

In [13]:
# !mkdir data
# !unzip {dataset_directory}/train.csv.zip -d data/
# !unzip {dataset_directory}/test.csv.zip  -d data/
# !unzip {dataset_directory}/test_labels.csv.zip  -d data/
# !unzip {dataset_directory}/sample_submission.csv.zip  -d data/

In [14]:
train_path = 'data/train.csv'
test_path = 'data/test.csv'
test_labels_path = 'data/test_labels.csv'
subm_path = 'data/sample_submission.csv'

In [15]:
# Importing the libraries needed
from tqdm.notebook import tqdm
import pandas as pd

In [16]:
# Setting up the device for GPU usage
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'

In [17]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_test_labels = pd.read_csv(test_labels_path)
df_test_labels = df_test_labels.set_index('id')

In [18]:
df_train.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [19]:
# hyperparameters
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 1e-05

# Get only 1024 samples as DEMO

In [20]:
df_train = df_train.sample(n=1024)

In [21]:
df_train.shape

(1024, 8)

# Train Test Split

In [22]:
train_size = 0.8
df_train_sampled = df_train.sample(frac=train_size, random_state=44)
df_val = df_train.drop(df_train_sampled.index).reset_index(drop=True)
df_train_sampled = df_train_sampled.reset_index(drop=True)
print()

In [23]:
df_train_sampled.shape, df_val.shape

((819, 8), (205, 8))

In [24]:
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

model_name = 'distilbert-base-uncased'

tokenizer = DistilBertTokenizer.from_pretrained(model_name, do_lower_case=True)

c:\Users\DINH QUOC BAO\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DINH QUOC BAO\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# Custom Dataset

In [25]:
class ToxicDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        # Initialize the attribute
        self.data = data # The dataset object
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.labels = self.data[label_cols].values

    def __len__(self):
        # Return the size of the dataset
        return len(self.data.id)
    
    def __getitem__(self, idx):
        # Return the i-th sample
        # Get the image and the label from the dataset object
        text = self.data.comment_text
        tokenized_text = self.tokenizer.encode_plus(
            str( text ),
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': tokenized_text['input_ids'].flatten(),
            'attention_mask': tokenized_text['attention_mask'].flatten(),
#             'token_type_ids': tokenized_text["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.labels[idx])
        }

In [26]:
train_dataset = ToxicDataset(df_train_sampled, tokenizer, MAX_LEN)
valid_dataset = ToxicDataset(df_val, tokenizer, MAX_LEN)

In [27]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

# Custom Model Class

In [28]:
# from transformers import DistilBertModel

class CustomDistilBertClass(torch.nn.Module):
    def __init__(self):
        super(CustomDistilBertClass, self).__init__()
        self.distilbert_model = DistilBertModel.from_pretrained(model_name, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 6)
    
    def forward(self, input_ids, attn_mask):
        output = self.distilbert_model(
            input_ids, 
            attention_mask=attn_mask, 
#             token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.last_hidden_state)
        output = self.linear(output_dropout)
        return output

In [29]:
model = CustomDistilBertClass()
model.to(device)
print()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
######### !! CHECK HERE ! ###########

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [31]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf

  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
#         print(f"batch_idx: {batch_idx}", sep=' ')
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask, )
        outputs = outputs[:, 0, :]
        targets = data['targets'].to(device, dtype = torch.float)
        loss = loss_fn(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)

            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, )
            outputs = outputs[:, 0, :]
            loss = loss_fn(outputs, targets)
            
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        
        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [32]:
import shutil

def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [34]:
ckpt_path = "C:/Users/DINH QUOC BAO/Documents/data_toxic/working/model.pt"
best_model_path = "C:/Users/DINH QUOC BAO/Documents/data_toxic/working/best_model.pt"

In [35]:
trained_model = train_model(EPOCHS, 
                            train_data_loader, 
                            val_data_loader, 
                            model, 
                            optimizer, 
                            ckpt_path, 
                            best_model_path)

############# Epoch 1: Training Start   #############


RuntimeError: CUDA out of memory. Tried to allocate 384.00 MiB (GPU 0; 6.00 GiB total capacity; 4.64 GiB already allocated; 0 bytes free; 4.68 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF